In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import statsmodels.api as sm
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt
import matplotlib.ticker
from haversine import haversine_vector, Unit
pd.options.mode.chained_assignment = None  # default='warn'
from sttn.data.lehd import OriginDestinationEmploymentDataProvider
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
provider = OriginDestinationEmploymentDataProvider()
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import scipy.optimize as optimize
import seaborn as sns

import math
from sttn.network import SpatioTemporalNetwork
from sttn.utils import add_distance
import os
%matplotlib inline

In [59]:

cities = [
    ('New York City',  [
        'Kings County, NY','Queens County, NY', 'New York County, NY','Bronx County, NY',
# 'Richmond County, NY','Westchester County, NY','Bergen County, NJ','Hudson County, NJ',
# 'Passaic County, NJ','Putnam County, NY',
#         'Rockland County, NY','Suffolk County, NY',
# 'Nassau County, NY','Middlesex County, NJ','Monmouth County, NJ','Ocean County, NJ',
# 'Somerset County, NJ','Essex County, NJ','Union County, NJ','Morris County, NJ',
# 'Sussex County, NJ','Hunterdon County, NJ','Pike County, PA'
    ]),
    
    ('Los Angeles', ['Los Angeles County, CA',
#                      'Orange County, CA',
#                      'San Bernardino County, CA', 'Riverside County, CA', 'Ventura County, CA'
                    ]),
    ('Chicago', [
        'Cook County, IL','DeKalb County, IL','DuPage County, IL','Grundy County, IL',
'Kankakee County, IL','Kane County, IL','Kendall County, IL','McHenry County, IL','Will County, IL',
#         'Jasper County, IN','Lake County, IN','Newton County, IN',
# 'Porter County, IN','Lake County, IL', 'Kenosha County, WI'
    ]),

    
    ('Dallas',  ['Collin County, TX','Dallas County, TX','Denton County, TX','Ellis County, TX',
                    'Hunt County, TX','Kaufman County, TX','Rockwall County, TX',
#                  'Johnson County, TX',
#                  'Parker County, TX','Tarrant County, TX','Wise County, TX'
                ]),
#     ('Atlanta', ['Barrow County, GA','Bartow County, GA', 'Butts County, GA','Carroll County, GA',
#                 'Cherokee County, GA', 'Clayton County, GA', 'Cobb County, GA', 'Coweta County, GA',
#                 'Dawson County, GA', 'DeKalb County, GA', 'Douglas County, GA', 'Fayette County, GA',
#                  'Forsyth County, GA', 'Fulton County, GA','Gwinnett County, GA', 'Haralson County, GA',
#                 'Heard County, GA', 'Henry County, GA', 'Jasper County, GA', 'Lamar County, GA',
#                  'Meriwether County, GA', 'Morgan County, GA', 'Newton County, GA', 'Paulding County, GA',
#                 'Pickens County, GA', 'Pike County, GA', 'Rockdale County, GA', 'Spalding County, GA', 
#                  'Walton County, GA']),
#     ('DC', ['Frederick County, MD','Montgomery County, MD', 'Washington, DC','Calvert County, MD',
#             'Charles County, MD', "Prince George's County, MD",' Arlington County, VA', 'Clarke County, VA',
#             'Culpeper County, VA', 'Fairfax County, VA', 'Fauquier County, VA', 'Loudoun County, VA',
#             'Prince William County, VA', 'Woodbridge, VA', 'Rappahannock County, VA', 'Spotsylvania County, VA',
#             'Stafford County, VA', 'Warren County, VA', 'Alexandria County, VA', 'Fairfax County, VA', 
#              'Falls Church County, VA', 'Fredericksburg County, VA', 'Manassas County, VA', 
#              'Manassas Park County, VA', 'Jefferson County, WV']),
#     ('San Francisco', ['Alameda County, CA', 'Contra Costa County, CA', 'Marin County, CA', 
#                        'San Francisco County, CA', 'San Mateo County, CA'])
]
     

In [60]:
def rwacbystate(state):
    if os.path.exists('racwac/%s.csv'%state):
        pass
    else:
        rac = pd.read_csv('https://lehd.ces.census.gov/data/lodes/LODES7/%s/rac/%s_rac_S000_JT00_2019.csv.gz'%(state,state),
                        compression='gzip')
        rac = rac[['h_geocode','C000','CE01','CE03']]
        rac = rac.rename(columns={'h_geocode':'ct','C000':'S000residence',
                                  'CE01':'SE01residence','CE03':'SE03residence'})
        wac = pd.read_csv('https://lehd.ces.census.gov/data/lodes/LODES7/%s/wac/%s_wac_S000_JT00_2019.csv.gz'%(state,state),
                        compression='gzip')
        wac = wac[['w_geocode','C000','CE01','CE03']]
        wac = wac.rename(columns={'w_geocode':'ct','C000':'S000jobs',
                                  'CE01':'SE01jobs','CE03':'SE03jobs'})
        df = rac.merge(wac,on='ct',how='outer').fillna(0)
        df.to_csv('racwac/%s.csv'%state,index=False)
        print(state,'rac, wac downloaded')

In [61]:
def xwalkbystate(state):
    if os.path.exists('xwalk/%s.csv'%state):
        pass
    else:
        xwalk = pd.read_csv('https://lehd.ces.census.gov/data/lodes/LODES7/%s/%s_xwalk.csv.gz'%(state,state),
                        compression='gzip')
        xwalk = xwalk[['tabblk2010','ctyname','blklatdd','blklondd']]
        xwalk = xwalk.rename(columns={'tabblk2010':'ct'})
        xwalk['ctyname'] = xwalk['ctyname'].apply(lambda x: x.split(',')[0])
        xwalk.to_csv('xwalk/%s.csv'%state,index=False)
        print(state,'xwalk downloaded')

In [62]:
def odbystate(state):
    if os.path.exists('od/%s.csv'%state):
        pass
    else:
        odaux = pd.read_csv('https://lehd.ces.census.gov/data/lodes/LODES7/%s/od/%s_od_aux_JT00_2019.csv.gz'%(state,state),
                        compression='gzip')
        odmain = pd.read_csv('https://lehd.ces.census.gov/data/lodes/LODES7/%s/od/%s_od_main_JT00_2019.csv.gz'%(state,state),
                        compression='gzip')
        od = pd.concat([odaux,odmain])
        od = od[['w_geocode', 'h_geocode','S000','SE01','SE02', 'SE03']]
        od = od.rename(columns={'w_geocode':'origin','h_geocode':'destination'})
        od.to_csv('od/%s.csv'%state,index=False)
        

In [64]:
target_columns = ['S000','SE01','SE03']
for city, counties in cities:
    print(city)
    odConcat = pd.DataFrame()
    wacracConcat = pd.DataFrame()
    xwalkConcat = pd.DataFrame()
    ctlst = []
    stateDict = {}
    
    # process county,state to a dict
    for county in counties:
        ctyname = county.split(',')[0]
        state = county.split(',')[1].strip().lower()
        stateDict[state] = stateDict.get(state,[]) + [ctyname]
    # read rac, wac, xwalk by state
    for state in stateDict.keys():
        print(state)
        rwacbystate(state)
        xwalkbystate(state)
        odbystate(state)
        statexwalk = pd.read_csv('xwalk/%s.csv'%state)
        statexwalk = statexwalk.loc[statexwalk['ctyname'].isin(stateDict[state])]
        ctlst += statexwalk['ct'].values.tolist()
        xwalkConcat = pd.concat([xwalkConcat,statexwalk],axis=0)
        
        
        od = pd.read_csv('od/%s.csv'%state)
        od.columns = ['origin', 'destination', 'S000flow', 'SE01flow', 'SE02flow', 'SE03flow']        
        odConcat = pd.concat([odConcat,od],axis=0)
        
#         wacrac = pd.read_csv('racwac/%s.csv'%state)
#         wacracConcat = pd.concat([wacracConcat,wacrac],axis=0)
        rac = odConcat.groupby(['origin']).agg({'S000flow':sum,'SE01flow':sum,'SE03flow':sum}).reset_index()
        rac.columns = ['ct','S000residence','SE01residence','SE03residence']
        wac = odConcat.groupby(['destination']).agg({'S000flow':sum,'SE01flow':sum,'SE03flow':sum}).reset_index()
        wac.columns = ['ct','S000jobs','SE01jobs','SE03jobs']
        wacracConcat = rac.merge(wac,on='ct',how='outer').fillna(0)
        
    odConcat = odConcat.loc[odConcat['origin'].isin(ctlst)]
    odConcat = odConcat.loc[odConcat['destination'].isin(ctlst)]
    print(odConcat.shape)
    # add rac, wac
    odConcat = odConcat.merge(wacracConcat[['ct','S000residence','SE01residence',
                                           'SE03residence']],left_on='origin',right_on='ct')
    del odConcat['ct']
    odConcat = odConcat.merge(wacracConcat[['ct','S000jobs','SE01jobs',
                                           'SE03jobs']],left_on='destination',right_on='ct')
    del odConcat['ct']
    print(odConcat.shape)
    
    # add lat, lon
    odConcat = odConcat.merge(xwalkConcat[['ct','blklatdd','blklondd']],left_on='origin',right_on='ct')
    odConcat = odConcat.rename(columns={'blklatdd':'olat','blklondd':'olng'})
    del odConcat['ct']
    odConcat = odConcat.merge(xwalkConcat[['ct','blklatdd','blklondd']],left_on='destination',right_on='ct')
    odConcat = odConcat.rename(columns={'blklatdd':'dlat','blklondd':'dlng'})
    del odConcat['ct']
    print(odConcat.shape)
    
    # calculate distance
    from_points = list(zip(odConcat.olat, odConcat.olng))
    to_points = list(zip(odConcat.dlat, odConcat.dlng))
    odConcat['distance'] = haversine_vector(from_points, to_points, Unit.KILOMETERS)
    
    # replace 0 distance by the shortest distance
    
#     odConcat.drop(['olat', 'olng', 'dlat', 'dlng'], axis=1, inplace=True)
    odConcat_above = odConcat.loc[odConcat.distance > 0]
    odConcat_below = odConcat.loc[odConcat.distance == 0]
    del odConcat_below['distance']
    misDistance = odConcat_above.groupby(['origin']).agg({'distance':min})
    misDistance['distance'] = misDistance['distance']
    odConcat_below = odConcat_below.merge(misDistance,on=['origin'])
    odConcat = pd.concat([odConcat_above,odConcat_below],axis=0)
    
    odConcat.to_csv('processedOD/%s.csv'%city)



New York City
ny
nj
(4070811, 6)
(4070811, 12)
(4070811, 16)
Los Angeles
ca
(3340227, 6)
(3340227, 12)
(3340227, 16)
Chicago
il
(3294527, 6)
(3294527, 12)
(3294527, 16)
Dallas
tx
(1741729, 6)
(1741729, 12)
(1741729, 16)


In [58]:
odConcat

,origin,destination,S000flow,SE01flow,SE02flow,SE03flow,S000residence,SE01residence,SE03residence,S000jobs,SE01jobs,SE03jobs,olat,olng,dlat,dlng,distance
0,60014001001007,60014001001044,1,0,0,1,55.0,24.0,25.0,71.0,8.0,51.0,37.868576,-122.235457,37.858686,-122.222436,1.586132
1,60014001001024,60014001001044,1,0,0,1,616.0,112.0,269.0,71.0,8.0,51.0,37.861152,-122.237696,37.858686,-122.222436,1.367431
2,60014001001054,60014001001044,1,0,0,1,11.0,2.0,3.0,71.0,8.0,51.0,37.855050,-122.226436,37.858686,-122.222436,0.535485
3,60014003004002,60014001001044,1,0,1,0,353.0,68.0,89.0,71.0,8.0,51.0,37.843227,-122.252215,37.858686,-122.222436,3.129034
4,60014012001017,60014001001044,1,0,0,1,258.0,34.0,161.0,71.0,8.0,51.0,37.832277,-122.256443,37.858686,-122.222436,4.188069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698,60816111003001,60816111003001,1,0,1,0,28.0,0.0,21.0,7.0,0.0,5.0,37.459668,-122.260633,37.459668,-122.260633,1.546022
1699,60411321002001,60411321002001,1,0,0,1,14.0,7.0,7.0,28.0,11.0,12.0,37.935678,-122.707519,37.935678,-122.707519,1.753266
1700,60816056004045,60816056004045,1,1,0,0,132.0,18.0,95.0,7.0,4.0,3.0,37.566808,-122.364661,37.566808,-122.364661,0.974017
1701,60816107001011,60816107001011,1,0,0,1,101.0,2.0,96.0,8.0,0.0,7.0,37.474010,-122.221453,37.474010,-122.221453,0.236083
